In [1]:
!pip install datasets
!pip install sentence_transformers
!pip install umap-learn
!pip install hdbscan
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("CShorten/ML-ArXiv-Papers")["train"]

# Extract abstracts to train on and corresponding titles
abstracts = dataset["abstract"][:1000]
titles = dataset["title"][:1000]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

ML-Arxiv-Papers.csv:   0%|          | 0.00/147M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/117592 [00:00<?, ? examples/s]

In [3]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(abstracts, show_progress_bar=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [4]:
from umap import UMAP

umap_model = UMAP(n_neighbors=7, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
# hyperparameters for UMAP

In [5]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=30, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
#hyperparameters for HDBSCAN


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english", min_df=1, ngram_range=(1, 2))
#hyperparameters for vectorizer

In [17]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
    "POS": pos_model
}


In [22]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(abstracts, embeddings)

# Show topics
topic_model.get_topic_info()

2024-11-21 06:36:08,599 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-21 06:36:13,598 - BERTopic - Dimensionality - Completed ✓
2024-11-21 06:36:13,604 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-21 06:36:13,676 - BERTopic - Cluster - Completed ✓
2024-11-21 06:36:13,693 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-21 06:36:31,853 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,427,-1_learning_algorithm_data_problem,"[learning, algorithm, data, problem, based, pa...","[learning, classifiers, machine learning, clas...","[learning, algorithm, paper, algorithms, metho...","[learning, algorithm, data, problem, paper, al...","[ In our previous work, we proposed a systema..."
1,0,153,0_learning_policy_regret_algorithm,"[learning, policy, regret, algorithm, problem,...","[reinforcement learning, bandit, reinforcement...","[policy, regret, reinforcement, optimal, reinf...","[learning, policy, regret, algorithm, problem,...",[ We consider an opportunistic spectrum acces...
2,1,105,1_learning_data_classification_feature,"[learning, data, classification, feature, kern...","[classifiers, feature selection, vector machin...","[classification, kernel, features, algorithm, ...","[learning, data, classification, feature, kern...",[ Kernel-based machine learning algorithms ar...
3,2,86,2_learning_distribution_algorithm_dimension,"[learning, distribution, algorithm, dimension,...","[sample complexity, learnability, learnable, c...","[dimension, eps, complexity, polynomial, distr...","[learning, distribution, dimension, function, ...",[ A $k$-modal probability distribution over t...
4,3,64,3_matrix_rank_sparse_low rank,"[matrix, rank, sparse, low rank, low, problem,...","[matrix completion, matrix factorization, comp...","[matrix, sparse, low rank, norm, collaborative...","[matrix, rank, sparse, low rank, low, problem,...",[ Low-rank matrix approximations are often us...
5,4,63,4_clustering_data_clusters_means,"[clustering, data, clusters, means, algorithms...","[clustering, clustering algorithms, cluster, s...","[clustering, clusters, algorithms, cluster, cl...","[clustering, data, clusters, means, algorithms...",[ Recent spectral clustering methods are a pr...
6,5,52,5_text_based_semantic_learning,"[text, based, semantic, learning, words, appro...","[text classification, information retrieval, c...","[text, semantic, classification, knowledge, wo...","[text, semantic, learning, words, approach, cl...",[ We participated in three of the protein-pro...
7,6,50,6_graph_model_algorithm_belief,"[graph, model, algorithm, belief, problem, mod...","[belief propagation, graphical models, graphic...","[algorithm, graphs, belief propagation, propag...","[graph, model, algorithm, belief, problem, mod...",[ In the context of inference with expectatio...


In [23]:
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

In [24]:
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
topic_model.visualize_documents(titles, reduced_embeddings=reduced_embeddings, custom_labels=True)

